In [1]:
import json
from collections import defaultdict

In [44]:
def get_verse_info(src_text):
    # Get Book with number of chapters and number of verses
    # dict of book name to array of numbers for # of verses
    with open(src_text) as f:
        text = json.load(f)
        info = defaultdict(lambda: defaultdict(list))
        for verse_info in text:
            book_name = verse_info['book_name']
            chap_num = int(verse_info['chapter_id'])
            info[book_name][chap_num].append(verse_info['verse_text'])
    
    return info

ot = get_verse_info('data/esv_ot.txt')
# for book_name, verses in ot.items():
#     print(book_name, len(verses))
nt = get_verse_info('data/esv_nt.txt')
# for book_name, verses in nt.items():
#     print(book_name, len(verses))
bible = {**ot, **nt}
# for book_name, verses in bible.items():
#     print(book_name, len(verses))

The rule is: if the line only consists of numbers, then the next line consists of the verse
If it starts with a number and then text, it will be that verse's text.
Maybe get the # of verse per chapter from ESV, and apply it here. Hard to distinguish between chapters in this text

In [19]:
def convert_nlt(bible_info, nlt_src, nlt_dest):
    with open(nlt_src, 'r') as src:
        lines = src.readlines()
        
    nlt = []
    # dict of book name to chapter to list of verses
    cur_book_name = ''
    book_names = []
    expected_book_idx = 0
    is_read_book_names = False
    current_verse = ""
    current_chapter = 1
    for line in lines:
        line = line.strip()
        if line == 'Book Names':
            is_read_book_names = True
        elif is_read_book_names:
            if line:
                book_names.append(line)
            else:
                is_read_book_names = False
        else: # Reading verses
            if expected_book_idx < len(book_names) and line == book_names[expected_book_idx]:
                if current_verse.strip():
                    nlt.append(current_verse.strip())
                cur_book_name = line
                current_chapter = 1
                expected_book_idx += 1
                current_verse = ''
            else:
                prefix_len = get_int_prefix(line)
                if prefix_len >= 0: # new verse
                    nlt.append(current_verse)
                    current_verse = line[prefix_len:]
                else:
                    current_verse += ' {}'.format(line)
                    
    with open(nlt_dest, 'w') as dest:
        for verse in nlt:
            dest.write(verse)
            dest.write('\n')

convert_nlt(bible, 'nlt_new.txt', 'nlt_something.txt') 

FileNotFoundError: [Errno 2] No such file or directory: 'nlt_new.txt'

In [122]:
def num_chapters(nlt, book_name):
    return len(nlt[book_name])

def num_verses(nlt, book_name, chapter):
    return len(nlt[book_name][chapter])

def read_nlt_something(bible_info, nlt_src):
    nlt = defaultdict(lambda : defaultdict(list))
    book_names = list(bible_info.keys())
    name_idx = 0
    chapter_idx = 0
    with open(nlt_src, 'r') as f:
        lines = f.readlines()
    
    line_idx = 0
    for book_name in book_names:
        for chapter_idx in range(num_chapters(bible_info, book_name)):
            for verse_idx in range(bible_info[book_name][chapter_idx]):
#                 if line_idx < len(lines):
                nlt[book_name][chapter_idx].append(lines[line_idx])
                line_idx += 1

    return book_names, nlt

book_names, nlt = read_nlt_something(bible, 'nlt_something.txt')

In [127]:
with open('data/nlt_final.txt', 'w') as f:
    json.dump(nlt, f)

In [47]:
def is_valid(gt, nlt):
    if len(gt) != len(nlt): # Number of books
        print(f'Exptected {len(gt)} # of books, but got {len(nlt)} # of books')
        return False
    
    for book_name in gt:
        if book_name not in nlt:
            print(f'{book_name} not found in NLT')
            return False
        if len(gt[book_name]) != len(nlt[book_name]):
            print(f'Expected {len(gt[book_name])} chapters for {book_name} but got {len(nlt[book_name])}')
            return False
        for chap_idx, verses in gt[book_name].items():
            if len(nlt[book_name][chap_idx]) != len(verses):
                print(f'Expected {len(verses)} verses for {book_name} {chap_idx} but got {len(nlt[book_name][chap_idx])} verses')
                return False
    
    return True

with open('data/nlt_final.txt') as f:
    data = json.load(f)
    nlt = defaultdict(lambda: defaultdict(list))
    count = 0
    for book_name, values in data.items():
        for chap_idx, verses in values.items():
            nlt[book_name][int(chap_idx) + 1] = verses
    print(is_valid(bible, nlt))

True
